# API 키 디버깅 노트북

API 키가 제대로 작동하는지 테스트합니다.

## 1. 환경 변수 로드 및 확인

In [ ]:
import sys
sys.path.append("..")

from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

# API 키 확인 (공백 제거)
fred_key = os.getenv("FRED_API_KEY", "").strip()
ecos_key = os.getenv("ECOS_API_KEY", "").strip()

print("="*80)
print("API 키 확인")
print("="*80)
print(f"FRED_API_KEY 존재: {bool(fred_key)}")
print(f"FRED_API_KEY 길이: {len(fred_key)}")
print(f"FRED_API_KEY 앞 5자리: {fred_key[:5] if fred_key else 'N/A'}...")
print()
print(f"ECOS_API_KEY 존재: {bool(ecos_key)}")
print(f"ECOS_API_KEY 길이: {len(ecos_key)}")
print(f"ECOS_API_KEY 앞 5자리: {ecos_key[:5] if ecos_key else 'N/A'}...")
print("="*80)

## 2. FRED API 테스트

In [ ]:
import requests
from datetime import datetime, timedelta

print("\n" + "="*80)
print("FRED API 테스트")
print("="*80)

if not fred_key:
    print("❌ FRED_API_KEY가 설정되지 않았습니다.")
else:
    # 간단한 요청
    url = "https://api.stlouisfed.org/fred/series/observations"
    
    # 최근 6개월 데이터 요청
    end_date = datetime.now()
    start_date = end_date - timedelta(days=180)
    
    params = {
        "series_id": "FEDFUNDS",
        "api_key": fred_key,
        "file_type": "json",
        "observation_start": start_date.strftime("%Y-%m-%d"),
        "frequency": "m"
    }
    
    print(f"요청 URL: {url}")
    print(f"시리즈: FEDFUNDS (미국 기준금리)")
    print(f"기간: {start_date.strftime('%Y-%m-%d')} ~ {end_date.strftime('%Y-%m-%d')}")
    print()
    
    try:
        response = requests.get(url, params=params)
        print(f"응답 상태 코드: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            observations = data.get("observations", [])
            
            if observations:
                print(f"✅ 성공! {len(observations)}개 데이터 수신")
                print("\n최근 3개 데이터:")
                for obs in observations[-3:]:
                    print(f"  날짜: {obs['date']}, 금리: {obs['value']}%")
            else:
                print("⚠️ 데이터가 없습니다.")
                print(f"응답: {data}")
        else:
            print(f"❌ 오류 발생")
            print(f"응답 내용: {response.text[:500]}")
            
    except Exception as e:
        print(f"❌ 예외 발생: {e}")

print("="*80)

## 3. ECOS API 테스트

In [ ]:
print("\n" + "="*80)
print("ECOS API 테스트")
print("="*80)

if not ecos_key:
    print("❌ ECOS_API_KEY가 설정되지 않았습니다.")
else:
    # 한국 기준금리 조회
    # 통계표코드: 722Y001 (한국은행 기준금리)
    
    end_date = datetime.now()
    start_date = end_date - timedelta(days=180)
    
    start_ym = start_date.strftime('%Y%m')
    end_ym = end_date.strftime('%Y%m')
    
    url = f"https://ecos.bok.or.kr/api/StatisticSearch/{ecos_key}/json/kr/1/100/722Y001/M/{start_ym}/{end_ym}"
    
    print(f"요청 URL: {url}")
    print(f"통계표: 722Y001 (한국은행 기준금리)")
    print(f"기간: {start_ym} ~ {end_ym}")
    print()
    
    try:
        response = requests.get(url)
        print(f"응답 상태 코드: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            
            # ECOS API 응답 구조 확인
            if "StatisticSearch" in data:
                rows = data["StatisticSearch"].get("row", [])
                
                if rows:
                    print(f"✅ 성공! {len(rows)}개 데이터 수신")
                    print("\n최근 3개 데이터:")
                    for row in rows[-3:]:
                        print(f"  날짜: {row['TIME']}, 금리: {row['DATA_VALUE']}%")
                else:
                    print("⚠️ 데이터가 없습니다.")
                    print(f"응답: {data}")
            else:
                print("❌ ECOS API 응답 형식 오류")
                print(f"응답: {data}")
        else:
            print(f"❌ 오류 발생")
            print(f"응답 내용: {response.text[:500]}")
            
    except Exception as e:
        print(f"❌ 예외 발생: {e}")

print("="*80)

## 4. InterestRateTool 클래스 테스트

In [ ]:
from tools.interest_rate_tool import InterestRateTool

print("\n" + "="*80)
print("InterestRateTool 테스트")
print("="*80)

# 도구 초기화 (디버깅 정보 출력됨)
rate_tool = InterestRateTool()

print("\n[1] 미국 기준금리 조회")
print("-" * 80)
us_data = rate_tool.get_us_rates("base_rate", months=6)

if us_data:
    print(f"\n최근 데이터: {us_data[-1]}")
else:
    print("\n데이터 조회 실패")

print("\n[2] 한국 기준금리 조회")
print("-" * 80)
kr_data = rate_tool.get_korea_rate(months=6)

if kr_data:
    print(f"\n최근 데이터: {kr_data[-1]}")
else:
    print("\n데이터 조회 실패")

print("\n" + "="*80)

## 5. 문제 해결 체크리스트

### ✅ 확인할 사항

1. **API 키가 .env 파일에 있는지**
   - `FRED_API_KEY=your_key` (공백 없이)
   - `ECOS_API_KEY=your_key` (공백 없이)

2. **API 키가 유효한지**
   - FRED: https://fred.stlouisfed.org/에서 확인
   - ECOS: https://ecos.bok.or.kr/에서 확인

3. **네트워크 연결**
   - 방화벽/VPN 확인

4. **Rate Limit**
   - FRED: 하루 120,000 요청
   - ECOS: 하루 10,000 요청

### 🔧 일반적인 오류

#### 400 Bad Request
- API 키가 잘못되었거나
- 요청 파라미터가 잘못됨

#### 404 Not Found
- URL이 잘못됨
- 시리즈 ID가 잘못됨

#### 401 Unauthorized
- API 키가 유효하지 않음
- API 키가 만료됨

## 6. 수동 API 키 테스트 (복사해서 실행)

API 키를 직접 입력해서 테스트해보세요.

In [ ]:
# 여기에 API 키를 직접 입력해서 테스트
test_fred_key = ""  # FRED API 키 입력
test_ecos_key = ""  # ECOS API 키 입력

if test_fred_key:
    url = f"https://api.stlouisfed.org/fred/series/observations?series_id=FEDFUNDS&api_key={test_fred_key}&file_type=json&limit=5"
    response = requests.get(url)
    print(f"FRED 테스트 - 상태 코드: {response.status_code}")
    if response.status_code == 200:
        print("✅ FRED API 정상 작동")
    else:
        print(f"❌ 오류: {response.text}")

if test_ecos_key:
    url = f"https://ecos.bok.or.kr/api/StatisticSearch/{test_ecos_key}/json/kr/1/10/722Y001/M/202401/202410"
    response = requests.get(url)
    print(f"\nECOS 테스트 - 상태 코드: {response.status_code}")
    if response.status_code == 200:
        print("✅ ECOS API 정상 작동")
    else:
        print(f"❌ 오류: {response.text}")